In [8]:
import numpy as np  
import matplotlib.pyplot as plt

data= np.loadtxt('beta_mean.txt')

In [10]:
data

array([1.37807681, 1.37547478, 1.37393279, 1.36312317, 1.37651263,
       1.37480787, 1.37450751, 1.37517025, 1.3833713 , 1.37441263,
       1.38562917, 1.37755262, 1.37405159, 1.37365787, 1.37874754,
       1.38003295, 1.37425503, 1.37291132, 1.38692632, 1.3789372 ,
       1.38345861, 1.38455001, 1.37424745, 1.38415174, 1.38288892,
       1.37669589, 1.38954658, 1.37696181, 1.36783112, 1.37376292,
       1.37045255, 1.36774026, 1.38333886, 1.36530018, 1.37938247,
       1.36694412, 1.37810334, 1.37367585, 1.36867543, 1.37385144,
       1.37522564, 1.37564095, 1.37033632, 1.37658319, 1.37249116,
       1.38445167, 1.378358  , 1.37982626, 1.37266822, 1.36958549,
       1.37615511, 1.36980424, 1.38326941, 1.37075175, 1.3697383 ,
       1.38200871, 1.37206488, 1.37320718, 1.37967645, 1.38083733,
       1.38416827, 1.37654183, 1.38061847, 1.37695061, 1.38450233,
       1.37967081, 1.37649304, 1.37395786, 1.37257783, 1.37625814,
       1.3841477 , 1.36945057, 1.3730696 , 1.36863548, 1.37746

In [24]:
import scipy.stats
from scipy.stats import norm

mean,var=scipy.stats.distributions.norm.fit(data)
boundary_1=norm.ppf(0.33, loc=mean, scale=var)
boundary_2=norm.ppf(0.66, loc=mean, scale=var)
# boundary_3=norm.ppf(0.60, loc=mean, scale=var)
# boundary_4=norm.ppf(0.60, loc=mean, scale=var)


zeros=np.zeros(1000)

y_zero=np.column_stack([data,zeros])
for i in range(0,len(y_zero[:,-1])):
    if y_zero[i,0]< boundary_1:
        y_zero[i,-1]=0
    elif boundary_1<y_zero[i,0]< boundary_2:
        y_zero[i,-1]=1
#     elif boundary_2<sigma_zero[i,3]< boundary_3:
#         sigma_zero[i,-1]=2
    else:
        y_zero[i,-1]=2
        
y=y_zero[:,-1]

# X_1=sigma_label[:,:4]
# y_1=sigma_label[:,-1].astype(int)

# # demonstrate data normalization with sklearn
# from sklearn.preprocessing import MinMaxScaler

# # create scaler
# scaler = MinMaxScaler()
# # fit and transform in one step
# normalized = scaler.fit_transform(X_1)

# from sklearn.neural_network import MLPClassifier
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(normalized, y_1,  test_size=0.3, train_size=0.7, stratify=y_1,
#                                                      random_state=1)
# clf = MLPClassifier(random_state=1, max_iter=2000).fit(X_train, y_train)
# clf.predict_proba(X_test[:1])


# clf.score(X_test, y_test)


In [25]:
x=np.loadtxt('beta_orientations.txt')

In [27]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel,
        random_state=0).fit(x, y)
gpc.score(x, y)

# gpc.predict_proba(x[:2,:])


0.343

In [30]:

	
# evaluate a gaussian process classifier model on the dataset
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.gaussian_process import GaussianProcessClassifier

# define model
model = GaussianProcessClassifier()
# define model evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, x, y, scoring='accuracy', cv=cv, n_jobs=-1)
# summarize result
print('Mean Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Mean Accuracy: 0.344 (0.018)


In [31]:
# grid search kernel for gaussian process classifier
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process.kernels import WhiteKernel

# define model
model = GaussianProcessClassifier()
# define model evaluation method
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
grid = dict()
grid['kernel'] = [1*RBF(), 1*DotProduct(), 1*Matern(),  1*RationalQuadratic(), 1*WhiteKernel()]
# define search
search = GridSearchCV(model, grid, scoring='accuracy', cv=cv, n_jobs=-1)
# perform the search
results = search.fit(x, y)
# summarize best
print('Best Mean Accuracy: %.3f' % results.best_score_)
print('Best Config: %s' % results.best_params_)
# summarize all
means = results.cv_results_['mean_test_score']
params = results.cv_results_['params']
for mean, param in zip(means, params):
    print(">%.3f with: %r" % (mean, param))

Best Mean Accuracy: 0.343
Best Config: {'kernel': 1**2 * WhiteKernel(noise_level=1)}
>0.328 with: {'kernel': 1**2 * RBF(length_scale=1)}
>0.318 with: {'kernel': 1**2 * DotProduct(sigma_0=1)}
>0.329 with: {'kernel': 1**2 * Matern(length_scale=1, nu=1.5)}
>0.328 with: {'kernel': 1**2 * RationalQuadratic(alpha=1, length_scale=1)}
>0.343 with: {'kernel': 1**2 * WhiteKernel(noise_level=1)}


In [32]:
np.max(data)

1.3959737011614728

In [33]:
np.min(data)

1.359078679909353

In [34]:
boundary_1

1.3730363445283618

In [35]:
boundary_2

1.377932967798732

In [38]:
np.max(data)-(np.max(data)-np.min(data))/3

1.3836753607440995

In [36]:
y

array([2., 1., 1., 0., 1., 1., 1., 1., 2., 1., 2., 1., 1., 1., 2., 2., 1.,
       0., 2., 2., 2., 2., 1., 2., 2., 1., 2., 1., 0., 1., 0., 0., 2., 0.,
       2., 0., 2., 1., 0., 1., 1., 1., 0., 1., 0., 2., 2., 2., 0., 0., 1.,
       0., 2., 0., 0., 2., 0., 1., 2., 2., 2., 1., 2., 1., 2., 2., 1., 1.,
       0., 1., 2., 0., 1., 0., 1., 1., 1., 1., 0., 2., 0., 0., 0., 1., 0.,
       1., 2., 2., 0., 0., 1., 2., 1., 0., 0., 1., 0., 2., 0., 2., 2., 1.,
       2., 2., 1., 2., 2., 0., 0., 1., 0., 2., 1., 0., 1., 0., 1., 1., 2.,
       2., 0., 2., 1., 2., 0., 1., 2., 1., 2., 2., 2., 0., 1., 1., 0., 0.,
       2., 2., 0., 2., 0., 2., 0., 2., 1., 1., 0., 1., 1., 0., 1., 1., 1.,
       0., 0., 0., 1., 0., 2., 2., 1., 1., 1., 0., 0., 0., 2., 0., 2., 0.,
       2., 0., 2., 0., 2., 1., 2., 2., 0., 2., 2., 0., 0., 0., 2., 2., 0.,
       0., 0., 1., 2., 0., 0., 2., 2., 1., 1., 2., 1., 1., 0., 1., 2., 2.,
       2., 2., 1., 2., 1., 0., 2., 2., 1., 2., 1., 2., 0., 2., 2., 1., 0.,
       0., 0., 0., 0., 1.